# Runner for Experiment 1


## Experiment

In [13]:
import pandas as pd
import matplotlib.pylab as o

from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.test.mock.backends import FakeManhattan
from qiskit.transpiler.passes import LookaheadSwap, StochasticSwap, SabreSwap
from qiskit.transpiler.passes import DenseLayout, ExVF2Layout, \
                                     NoiseAdaptiveLayout, SabreLayout

from exp.custom_passmanager import custom_pass_manager
from exp.circuit_generators import graphstate_complete, graphstate_manhattan_ring

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
def exp1(circuit, layouter, swapper, backend, layouter_reps=100):
    passmanager = custom_pass_manager(backend, layouter, swapper, layout_reps=layouter_reps)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if 'Swap' in kwargs['pass_'].name():
            count_ops_after_map.update(kwargs['dag'].count_ops())
            
    
    transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']
    time = times[layouter.name()] / layouter_reps

    return time, needed_swaps

## Execution

In [17]:
layouter_reps = 100
backend = FakeManhattan()
backend_properties = backend.properties()
coupling_map = CouplingMap(backend.configuration().coupling_map)

#swapper = StochasticSwap(coupling_map, trials=20, seed=0)
swapper = SabreSwap(coupling_map, heuristic="decay", seed=0)

layout_type_dict = {
    'exvf2': ExVF2Layout(coupling_map), 
    'dense': DenseLayout(coupling_map, backend_properties), 
    #'noise_adaptive': NoiseAdaptiveLayout(backend_properties),
    'sabre': SabreLayout(coupling_map, max_iterations=1)
}
circuit_type_dict = {
    'full': graphstate_complete(65),
    '12q_ring': graphstate_manhattan_ring(12),
    '12q_full': graphstate_complete(12)
}

In [18]:
df = pd.DataFrame(columns=["layouter", "circuit", "time", "swaps"])

# loop over layouters
for layouter_name, layouter in layout_type_dict.items():

    # loop over circuits
    for circuit_name, circuit in circuit_type_dict.items():

        time, swaps = exp1(circuit, layouter, swapper, backend, layouter_reps)
        result = {'layouter': layouter_name,
                  'circuit': circuit_name,
                  'time': time,
                  'swaps': swaps}
        df = df.append(result, ignore_index=True)

In [19]:
print(df)

  layouter   circuit      time swaps
0    exvf2      full  0.000228  3399
1    exvf2  12q_ring  0.000122     0
2    exvf2  12q_full  0.821010   101
3    dense      full  0.002918  3467
4    dense  12q_ring  0.000641     4
5    dense  12q_full  0.000641   108
6    sabre      full  0.144383  2876
7    sabre  12q_ring  0.000421    22
8    sabre  12q_full  0.002315    70
